In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 714 kB/s 
     |████████████████████████████████| 6.4 MB 41.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
            unattached_dfs.append(process(pd.read_csv(address)))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [20]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [23]:
clmns = list(range(1, 45))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i):
  mean_of_next = []
  for ix in range(i,i+10):
    mean_of_next.append(data[ix][0]+data[ix][3])
  mean_of_next = [element / 2 for element in mean_of_next]
  mean_of_next = sum(mean_of_next)/10
  mean_of_previous = []
  for ix in range(i-10,i):
    mean_of_previous.append(data[ix][0]+data[ix][3])
  mean_of_previous = [element / 2 for element in mean_of_previous]
  mean_of_previous = sum(mean_of_previous)/10
  if mean_of_next > mean_of_previous:
    return "buy"
  else:
    return "sell"
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(132, data.shape[0]-10):
        opens = []
        highs = []
        lows = []
        closes = []           
        for ix in range(132, 0, -6):
            opens.append([(data[i-ix][0]+data[i-ix+1][0]+data[i-ix+2][0]) /3 - (data[i-ix+3][0]+data[i-ix+4][0]+data[i-ix+5][0])/3])
            closes.append([(data[i-ix][3]+data[i-ix+1][3]+data[i-ix+2][3]) /3 - (data[i-ix+3][3]+data[i-ix+4][3]+data[i-ix+5][3])/3])
        sugg = make_y(data, i)
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)


In [24]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1499
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240  
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

In [26]:
yTrain.value_counts()

suggestion_buy  suggestion_sell
1               0                  32874
0               1                  31343
dtype: int64

In [27]:
model = Sequential()

model.add(Dense(750, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 750)               33750     
                                                                 
 dense_16 (Dense)            (None, 750)               563250    
                                                                 
 dense_17 (Dense)            (None, 750)               563250    
                                                                 
 dense_18 (Dense)            (None, 750)               563250    
                                                                 
 dense_19 (Dense)            (None, 2)                 1502      
                                                                 
Total params: 1,725,002
Trainable params: 1,725,002
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=25,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"MCG136_{acr}.h5")

Epoch 1/25
2007/2007 [==============================] - 46s 22ms/step - loss: 0.3628 - accuracy: 0.8399 - val_loss: 0.3044 - val_accuracy: 0.8707
Epoch 2/25
2007/2007 [==============================] - 44s 22ms/step - loss: 0.2420 - accuracy: 0.8987 - val_loss: 0.2009 - val_accuracy: 0.9256
Epoch 3/25
2007/2007 [==============================] - 44s 22ms/step - loss: 0.1638 - accuracy: 0.9356 - val_loss: 0.1538 - val_accuracy: 0.9400
Epoch 4/25
2007/2007 [==============================] - 44s 22ms/step - loss: 0.1232 - accuracy: 0.9534 - val_loss: 0.1556 - val_accuracy: 0.9423
Epoch 5/25
2007/2007 [==============================] - 44s 22ms/step - loss: 0.1016 - accuracy: 0.9626 - val_loss: 0.1237 - val_accuracy: 0.9558
Epoch 6/25
 545/2007 [=======>......................] - ETA: 29s - loss: 0.0839 - accuracy: 0.9695

In [ ]:
model.fit(xTrain,yTrain,epochs=25,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"MCG136_{acr}.h5")

Epoch 1/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.3997 - accuracy: 0.8197 - val_loss: 0.3452 - val_accuracy: 0.8523
Epoch 2/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.2731 - accuracy: 0.8853 - val_loss: 0.2566 - val_accuracy: 0.8946
Epoch 3/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.1845 - accuracy: 0.9274 - val_loss: 0.2039 - val_accuracy: 0.9216
Epoch 4/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.1389 - accuracy: 0.9468 - val_loss: 0.1689 - val_accuracy: 0.9382
Epoch 5/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.1123 - accuracy: 0.9578 - val_loss: 0.1540 - val_accuracy: 0.9488
Epoch 6/25
2994/2994 [==============================] - 58s 19ms/step - loss: 0.0949 - accuracy: 0.9653 - val_loss: 0.1430 - val_accuracy: 0.9550
Epoch 7/25
2994/2994 [==============================] - 57s 19ms/step - loss: 0.0822 - accuracy: 0.9689 - val_loss: 0.1293 -

In [ ]:
import plotly.graph_objects as go
df = yf.download("btc-usd",period="max",interval="1d")
df = df.tail(50)
candlestick = go.Candlestick( x=df.index,open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])
fig = go.Figure(data=[candlestick])
fig.add_trace(
    go.Scatter(
        x=["2022-5-10"],
        y=[35555],
        mode="markers+text",
        marker=dict(symbol='triangle-down-open', size = 12),

    )
)
fig.show()

[*********************100%***********************]  1 of 1 completed


In [ ]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        opens = []
        highs = []
        lows = []
        closes = []
        opens.append([
                      (data[i-96][0]+data[i-95][0]+data[i-94][0])/3 - (data[i-93][0]+data[i-92][0]+data[i-91][0])/3 , (data[i-90][0]+data[i-89][0]+data[i-88][0])/3 - (data[i-87][0]+data[i-86][0]+data[i-85][0])/3,
                      (data[i-84][0]+data[i-83][0]+data[i-82][0])/3 - (data[i-81][0]+data[i-80][0]+data[i-79][0])/3 , (data[i-78][0]+data[i-77][0]+data[i-76][0])/3 - (data[i-75][0]+data[i-74][0]+data[i-73][0])/3,
                      (data[i-72][0]+data[i-71][0]+data[i-70][0])/3 - (data[i-69][0]+data[i-68][0]+data[i-67][0])/3 , (data[i-66][0]+data[i-65][0]+data[i-64][0])/3 - (data[i-63][0]+data[i-62][0]+data[i-61][0])/3,
                      (data[i-60][0]+data[i-59][0]+data[i-58][0])/3 - (data[i-57][0]+data[i-56][0]+data[i-55][0])/3 , (data[i-54][0]+data[i-53][0]+data[i-52][0])/3 - (data[i-51][0]+data[i-50][0]+data[i-49][0])/3,
                      (data[i-48][0]+data[i-47][0]+data[i-46][0])/3 - (data[i-45][0]+data[i-44][0]+data[i-43][0])/3 , (data[i-42][0]+data[i-41][0]+data[i-40][0])/3 - (data[i-39][0]+data[i-38][0]+data[i-37][0])/3,
                      (data[i-36][0]+data[i-35][0]+data[i-34][0])/3 - (data[i-33][0]+data[i-32][0]+data[i-31][0])/3 , (data[i-30][0]+data[i-29][0]+data[i-28][0])/3 - (data[i-27][0]+data[i-26][0]+data[i-25][0])/3,
                      (data[i-24][0]+data[i-23][0]+data[i-22][0])/3 - (data[i-21][0]+data[i-20][0]+data[i-19][0])/3 , (data[i-18][0]+data[i-17][0]+data[i-16][0])/3 - (data[i-15][0]+data[i-14][0]+data[i-13][0])/3,
                      (data[i-12][0]+data[i-11][0]+data[i-10][0])/3 - (data[i-9][0]+data[i-8][0]+data[i-7][0])/3    , (data[i-6][0]+data[i-5][0]+data[i-4][0])/3    - (data[i-3][0]+data[i-2][0]+data[i-1][0])/3])
        closes.append([
                      (data[i-96][3]+data[i-95][3]+data[i-94][3])/3 - (data[i-93][3]+data[i-92][3]+data[i-91][3])/3 , (data[i-90][3]+data[i-89][3]+data[i-88][3])/3 - (data[i-87][3]+data[i-86][3]+data[i-85][3])/3,
                      (data[i-84][3]+data[i-83][3]+data[i-82][3])/3 - (data[i-81][3]+data[i-80][3]+data[i-79][3])/3 , (data[i-78][3]+data[i-77][3]+data[i-76][3])/3 - (data[i-75][3]+data[i-74][3]+data[i-73][3])/3,
                      (data[i-72][3]+data[i-71][3]+data[i-70][3])/3 - (data[i-69][3]+data[i-68][3]+data[i-67][3])/3 , (data[i-66][3]+data[i-65][3]+data[i-64][3])/3 - (data[i-63][3]+data[i-62][3]+data[i-61][3])/3,
                      (data[i-60][3]+data[i-59][3]+data[i-58][3])/3 - (data[i-57][3]+data[i-56][3]+data[i-55][3])/3 , (data[i-54][3]+data[i-53][3]+data[i-52][3])/3 - (data[i-51][3]+data[i-50][3]+data[i-49][3])/3,  
                      (data[i-48][3]+data[i-47][3]+data[i-46][3])/3 - (data[i-45][3]+data[i-44][3]+data[i-43][3])/3 , (data[i-42][3]+data[i-41][3]+data[i-40][3])/3 - (data[i-39][3]+data[i-38][3]+data[i-37][3])/3,
                      (data[i-36][3]+data[i-35][3]+data[i-34][3])/3 - (data[i-33][3]+data[i-32][3]+data[i-31][3])/3 , (data[i-30][3]+data[i-29][3]+data[i-28][3])/3 - (data[i-27][3]+data[i-26][3]+data[i-25][3])/3,
                      (data[i-24][3]+data[i-23][3]+data[i-22][3])/3 - (data[i-21][3]+data[i-20][3]+data[i-19][3])/3 , (data[i-18][3]+data[i-17][3]+data[i-16][3])/3 - (data[i-15][3]+data[i-14][3]+data[i-13][3])/3,
                      (data[i-12][3]+data[i-11][3]+data[i-10][3])/3 - (data[i-9][3]+data[i-8][3]+data[i-7][3])/3    , (data[i-6][3]+data[i-5][3]+data[i-4][3])/3    - (data[i-3][3]+data[i-2][3]+data[i-1][3])/3])

        mean_of_previous = []
        for ix in range(i-5,i):
           mean_of_previous.append(data[ix][0]+data[ix][3])
        mean_of_previous = [element / 2 for element in mean_of_previous]
        mean_of_previous = sum(mean_of_previous)/5
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1))
        return arr,mean_of_previous
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index)
    return f"""YF   : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=105), tindex)
    return f"""TVB  : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [ ]:
make_prediction("btc-usd","max","1wk","btcusd","bitstamp",Interval.in_weekly,-1,-2)

[*********************100%***********************]  1 of 1 completed


ValueError: ignored

In [ ]:
yTest

In [ ]:
yf.download("btc-usd",period="max",interval="1mo")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')